In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [71]:
data = pd.read_csv('data_v3/train_data_v3.csv', parse_dates=True)

In [72]:
data

,time,Measure,kWh,label
0,2022-01-03 00:00:00,Mech_Svc_AHU,8.0000,unclassified
1,2022-01-03 00:00:00,Sub - Main M2,1000.0000,unclassified
2,2022-01-03 00:00:00,FB1 - Main 1L,26.5000,unclassified
3,2022-01-03 00:00:00,AB3 - Main 3L,36.7500,Non-production
4,2022-01-03 00:00:00,Sub - Feeder F04,0.0000,unclassified
...,...,...,...,...
652419,2022-11-13 23:45:00,Sub - Feeder F03,22.1250,unclassified
652420,2022-11-13 23:45:00,Sub - Feeder F04,0.0000,unclassified
652421,2022-11-13 23:45:00,Paint_AHU,8.2500,unclassified
652422,2022-11-13 23:45:00,BS4 - Main 4L,47.5000,unclassified


In [75]:
data.drop(data[data['label'] == 'unclassified'].index, inplace=True)

In [76]:
data

,time,Measure,kWh,label
3,2022-01-03 00:00:00,AB3 - Main 3L,36.7500,Non-production
5,2022-01-03 00:00:00,Sub - Feeder F02,64.2500,Non-production
8,2022-01-03 00:00:00,BS4 - Main 4R,19.2500,Non-production
12,2022-01-03 00:00:00,BS1 - Main 1L,31.7500,Non-production
24,2022-01-03 00:15:00,BS4 - Main 4R,19.2500,Non-production
...,...,...,...,...
652391,2022-11-13 23:30:00,BS1 - Main 1L,34.6250,Non-production
652392,2022-11-13 23:30:00,AB3 - Main 3L,30.8750,Non-production
652404,2022-11-13 23:45:00,BS1 - Main 1L,34.3125,Non-production
652405,2022-11-13 23:45:00,BS4 - Main 4R,30.8750,Non-production


In [77]:
data1 = pd.get_dummies(data, columns=['label'])
data1

,time,Measure,kWh,label_Non-production,label_Power-down,label_Power-up,label_Production
3,2022-01-03 00:00:00,AB3 - Main 3L,36.7500,1,0,0,0
5,2022-01-03 00:00:00,Sub - Feeder F02,64.2500,1,0,0,0
8,2022-01-03 00:00:00,BS4 - Main 4R,19.2500,1,0,0,0
12,2022-01-03 00:00:00,BS1 - Main 1L,31.7500,1,0,0,0
24,2022-01-03 00:15:00,BS4 - Main 4R,19.2500,1,0,0,0
...,...,...,...,...,...,...,...
652391,2022-11-13 23:30:00,BS1 - Main 1L,34.6250,1,0,0,0
652392,2022-11-13 23:30:00,AB3 - Main 3L,30.8750,1,0,0,0
652404,2022-11-13 23:45:00,BS1 - Main 1L,34.3125,1,0,0,0
652405,2022-11-13 23:45:00,BS4 - Main 4R,30.8750,1,0,0,0


In [78]:
data.label.unique()

array(['Non-production', 'Power-up', 'Production', 'Power-down'],
      dtype=object)

In [79]:
d = {'unclassified':0, 'Non-production':1, 'Power-up':2, 'Production':3, 'Power-down':4}

In [80]:
data.replace({'label': d}, inplace=True)

In [81]:
X = data1.drop(['time', 'Measure'], axis=1).values.astype('float32')
y = data['label'].values.astype('float32')

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45, random_state=101)

In [84]:
print(X_train.shape)
print(y_train.shape)

(44721, 5)
(44721,)


In [88]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense

In [89]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=1, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
2796/2796 [==============================] - 3s 925us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 2/10
2796/2796 [==============================] - 2s 891us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 3/10
2796/2796 [==============================] - 2s 884us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 4/10
2796/2796 [==============================] - 2s 893us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 5/10
2796/2796 [==============================] - 3s 941us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 6/10
2796/2796 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 7/10
2796/2796 [==============================] - 3s 912us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 8/10
2796/2796 [==============================] - 3s 901us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 9/10
2796/2796 [==============================] - 3s 903us/step - loss: 0.0000e+00 - accuracy: 0.4311
Epoch 10/10
2796/2796 [=======

In [90]:
loss, accuracy = model.evaluate(x=X_test, y=y_test, batch_size=5)

print("Test loss:", loss)
print("Test accuracy:", accuracy)

7319/7319 [==============================] - 6s 797us/step - loss: 0.0000e+00 - accuracy: 0.4222
Test loss: 0.0
Test accuracy: 0.42223498225212097


In [91]:
final = pd.read_csv('data_v3/holdout_v3.csv', parse_dates=True)

In [92]:
final

,time,Measure,kWh,label
0,2020-07-27 00:00:00,Sub - Feeder F08,481.500,Non-production
1,2020-07-27 00:15:00,Sub - Feeder F08,481.375,Non-production
2,2020-07-27 00:30:00,Sub - Feeder F08,481.250,Non-production
3,2020-07-27 00:45:00,Sub - Feeder F08,481.125,Non-production
4,2020-07-27 01:00:00,Sub - Feeder F08,481.000,Non-production
...,...,...,...,...
2683,2020-09-20 22:45:00,Sub - Feeder F08,372.125,Non-production
2684,2020-09-20 23:00:00,Sub - Feeder F08,371.500,Non-production
2685,2020-09-20 23:15:00,Sub - Feeder F08,378.125,Non-production
2686,2020-09-20 23:30:00,Sub - Feeder F08,384.750,Non-production


In [93]:
final.label.unique()

array(['Non-production', 'Power-up', 'Production', 'Power-down'],
      dtype=object)

In [94]:
final.replace({'label': d}, inplace=True)

In [95]:
final1 = pd.get_dummies(data=final, columns=['label'])
final1.head()

,time,Measure,kWh,label_1,label_2,label_3,label_4
0,2020-07-27 00:00:00,Sub - Feeder F08,481.500,1,0,0,0
1,2020-07-27 00:15:00,Sub - Feeder F08,481.375,1,0,0,0
2,2020-07-27 00:30:00,Sub - Feeder F08,481.250,1,0,0,0
3,2020-07-27 00:45:00,Sub - Feeder F08,481.125,1,0,0,0
4,2020-07-27 01:00:00,Sub - Feeder F08,481.000,1,0,0,0


In [96]:
X_test = final1.drop(['time', 'Measure'], axis=1).values.astype('float32')
y_test = final['label'].values.astype('float32')

In [97]:
print(X_test.shape)
print(y_test.shape)

(2688, 5)
(2688,)


In [98]:
model.predict(X_test, batch_size=12)

224/224 [==============================] - 0s 702us/step


array([[0.99999994],
       [0.99999994],
       [0.99999994],
       ...,
       [1.        ],
       [1.        ],
       [1.        ]], dtype=float32)

In [106]:
y_predict = model.predict(X_test, batch_size=12)

224/224 [==============================] - 0s 718us/step


In [99]:
from sklearn.metrics import confusion_matrix, f1_score

In [113]:
print(y_predict.shape)
print(y_test.shape)

(2688, 1)
(2688,)


In [ ]:
def take_path():
    return test_set = pd.read_csv('test_data/test_data/test_Body_AHU.csv')